In [39]:
import numpy as np
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
from datetime import timezone, timedelta
from scipy.signal import argrelextrema
from itertools import product
import pandas as pd
import seaborn as sns

In [40]:
log_fname = "/home/sparrow/soft/transloc/log.tsv"

In [41]:
format = '%I:%M%p'

broadway_times_weekday = ["6:00am","6:15am","6:30am","6:45am","7:00am","7:15am","7:30am","7:36am","7:42am",
                          "7:48am","7:54am","8:00am","8:06am","8:12am","8:18am","8:24am","8:30am","8:36am",
                          "8:42am","8:48am","8:54am","9:00am","9:10am","9:20am","9:30am","9:40am","9:50am",
                          "10:00am","10:15am","10:30am","10:45am","11:00am","11:15am","11:30am","11:45am",
                          "12:00pm","12:15pm","12:30pm","12:45pm","1:00pm","1:15pm","1:30pm","1:45pm","2:00pm",
                          "2:15pm","2:30pm","2:45pm","3:00pm","3:06pm","3:12pm","3:18pm","3:24pm","3:30pm",
                          "3:36pm","3:42pm","3:48pm","3:54pm","4:00pm","4:06pm","4:12pm","4:18pm","4:24pm",
                          "4:30pm","4:36pm","4:42pm","4:48pm","4:54pm","5:00pm","5:06pm","5:12pm","5:18pm",
                          "5:24pm","5:30pm","5:36pm","5:42pm","5:48pm","5:54pm","6:00pm","6:10pm","6:20pm",
                          "6:30pm","6:45pm","7:00pm","7:15pm","7:30pm","7:45pm","8:00pm","8:15pm","8:30pm",
                          "9:00pm","9:30pm","10:00pm","10:30pm","11:00pm","11:30pm","12:00am","12:30am"]
interfaith_times_weekday = ["6:00am","6:15am","6:30am","6:45am","7:00am","7:15am","7:30am","7:36am","7:42am",
                            "7:48am","7:54am","8:00am","8:06am","8:12am","8:18am","8:24am","8:30am","8:36am",
                            "8:42am","8:48am","8:54am","9:00am","9:06am","9:12am","9:18am","9:24am","9:30am",
                            "9:40am","9:50am","10:00am","10:10am","10:20am","10:30am","10:45am","11:00am",
                            "11:15am","11:30am","11:45am","12:00pm","12:15pm","12:30pm","12:45pm","1:00pm",
                            "1:15pm","1:30pm","1:45pm","2:00pm","2:15pm","2:30pm","2:45pm","3:00pm","3:15pm",
                            "3:30pm","3:36pm","3:42pm","3:48pm","3:54pm","4:00pm","4:06pm","4:12pm","4:18pm",
                            "4:24pm","4:30pm","4:36pm","4:42pm","4:48pm","4:54pm","5:00pm","5:06pm","5:12pm",
                            "5:18pm","5:24pm","5:30pm","5:36pm","5:42pm","5:48pm","5:54pm","6:00pm","6:10pm",
                            "6:20pm","6:30pm","6:40pm","6:50pm","7:00pm","7:15pm","7:30pm","7:45pm","8:00pm",
                            "8:15pm","8:30pm","8:45pm","9:00pm","9:30pm","10:00pm","10:30pm","11:00pm","11:30pm",
                            "12:00am","12:30am"]
times_sat = ["7:25am","7:50am","8:15am","8:40am","9:05am","9:30am","9:55am","10:20am","10:45am","11:10am",
             "11:35am","12:00pm","12:25pm","12:50pm","1:15pm","1:40pm","2:05pm","2:30pm","2:55pm","3:20pm",
             "3:45pm","4:10pm","4:35pm","5:00pm","5:25pm","5:50pm","6:15pm","6:40pm","7:05pm","7:30pm",
             "7:55pm","8:20pm","8:45pm","9:10pm","9:35pm","10:00pm","10:25pm","10:50pm","11:15pm","11:40pm",
             "12:05am","12:30am"]
times_sun = ["11:10am","11:35am","12:00pm","12:25pm","12:50pm","1:15pm","1:40pm","2:05pm","2:30pm","2:55pm",
             "3:20pm","3:45pm","4:10pm","4:35pm","5:00pm","5:25pm","5:50pm","6:15pm","6:40pm","7:05pm",
             "7:30pm","7:55pm","8:20pm","8:45pm","9:10pm","9:35pm","10:00pm","10:25pm","10:50pm","11:15pm",
             "11:40pm"]

In [136]:
log = pd.read_csv(log_fname,sep="\t",names=["sid","vid","timestamp","dist"])
log["time"] = pd.to_datetime(log["timestamp"],unit='ms',utc=True).dt.tz_convert("US/Eastern")
print(len(log))

864750


In [137]:
n=100
max_time_diff = 1200000 # 20 minutes in unix time
min_max_distance_between_stops = 3500 # distance between the two stops
min_distance_to_stop = 250

name2sid = {"Broadway":4177254,
            "Interfaith":4265754}
sid2name = {4177254:"Broadway",
            4265754:"Interfaith"}

stops = {4177254:{"name":"Broadway",
                  "times":{"weekday":[datetime.time(datetime.strptime(x,format)) for x in broadway_times_weekday],
                           "sat":[datetime.time(datetime.strptime(x,format)) for x in times_sat],
                           "sun":[datetime.time(datetime.strptime(x,format)) for x in times_sun]}},
         4265754:{"name":"Interfaith",
                  "times":{"weekday":[datetime.time(datetime.strptime(x,format)) for x in interfaith_times_weekday],
                           "sat":[datetime.time(datetime.strptime(x,format)) for x in times_sat],
                           "sun":[datetime.time(datetime.strptime(x,format)) for x in times_sun]}}}

In [138]:
days = dict()

all_stops_lst = []

glog = log.groupby(by=["sid","vid"])
grp = ""
for name,grp in glog:
    # Find local peaks
#     grp['min'] = grp.dist[(grp.dist.shift(1) > grp.dist) & (grp.dist.shift(-1) > grp.dist) & (grp.dist<=100)]
    
    grp['min'] = grp.iloc[argrelextrema(grp.dist.values, np.less_equal,
                    order=n)[0]]['dist']
    grp["min"] = grp[~(grp["min"].isna())&(grp["min"]<min_distance_to_stop)]["min"]
    
    
    # remove duplicates close in time
    sub = grp[~(grp["min"].isna())].reset_index(drop=True)

    sub["group_diff"] = sub.sort_values("timestamp")["timestamp"]\
                         .diff()\
                         .gt(max_time_diff)\
                         .cumsum()

    sub_grp_diff = pd.DataFrame(sub.groupby(by="group_diff").max()).reset_index()
    grp["min"] = grp[~(grp["min"].isna())&(grp["timestamp"].isin(sub_grp_diff["timestamp"]))]["min"]
    
    
    # we probably should also make sure that the bus actually went somewhere meaningful 
    # and not just circled around without passengers
    
    # can be done as:
    # compute distance between the two endpoints
    # get maximum distance between two inferred stops
    # if not within n% of the total route distance
    
    idxs = list(grp[~(grp["min"].isna())].index)
    
    good_idxs = []
    
    if len(idxs)>=1:
        good_idxs.append(idxs[0])

    if len(idxs)>1:
        for i in idxs[1:]:
            max_dist_between = grp.loc[good_idxs[-1]:i]["dist"].max()
            if max_dist_between<=min_max_distance_between_stops:
                good_idxs[-1] = i
            else:
                good_idxs.append(i)
                
    grp["min"] = grp[~(grp["min"].isna())&(grp.index.isin(good_idxs))]["min"]
    min_dist = grp[~(grp["min"].isna())&(grp.index.isin(good_idxs))]["min"].iloc[0]
    grp["time"] = pd.to_datetime(grp[~(grp["min"].isna())]["timestamp"],unit='ms',utc=True).dt.tz_convert("US/Eastern")
    
    # add times
    for t in list(grp[~(grp["min"].isna())]["time"]):
        if t.date() not in days:
            if t.weekday() <= 4: # weekday
                days[t.date()] = {name2sid["Broadway"]:{"scheduled":[x for x in stops[name2sid["Broadway"]]["times"]["weekday"]],
                                                        "observed":[]},
                                  name2sid["Interfaith"]:{"scheduled":[x for x in stops[name2sid["Interfaith"]]["times"]["weekday"]],
                                                          "observed":[]}}
            else:
                if t.weekday() == 5: # Saturday
                    days[t.date()] = {name2sid["Broadway"]:{"scheduled":[x for x in stops[name2sid["Broadway"]]["times"]["sat"]],
                                                            "observed":[]},
                                      name2sid["Interfaith"]:{"scheduled":[x for x in stops[name2sid["Interfaith"]]["times"]["sat"]],
                                                              "observed":[]}}
                if t.weekday() == 6: # Sunday
                    days[t.date()] = {name2sid["Broadway"]:{"scheduled":[x for x in stops[name2sid["Broadway"]]["times"]["sun"]],
                                                            "observed":[]},
                                      name2sid["Interfaith"]:{"scheduled":[x for x in stops[name2sid["Interfaith"]]["times"]["sun"]],
                                                              "observed":[]}}

        # add days
        days[t.date()][name[0]]["observed"].append(t.time())
        all_stops_lst.append([datetime.combine(t.date(),t.time()),min_dist,sid2name[name[0]],name[1]])

In [139]:
observed_df = pd.DataFrame(all_stops_lst)
observed_df.columns = ["time","dist","stop","vid"]


observed_df = observed_df[~(observed_df['time'].dt.date==min(observed_df['time'].dt.date))&
                        ~(observed_df['time'].dt.date==max(observed_df['time'].dt.date))].reset_index(drop=True)

observed_df.to_csv(log_fname+".observed.tsv",sep="\t",index=False)
observed_df.head()

,time,dist,stop,vid
0,2022-11-12 07:50:42,2.666306,Broadway,4019313
1,2022-11-12 08:42:06,2.666306,Broadway,4019313
2,2022-11-12 09:32:24,2.666306,Broadway,4019313
3,2022-11-12 10:22:00,2.666306,Broadway,4019313
4,2022-11-12 11:24:06,2.666306,Broadway,4019313


In [140]:
# recursive function to find closest times
# 1. find the closest pair from the two lists that meets criteria
# 2. return list without the two timepoints

def get_delta(t1,t2):
    ct1 = datetime.combine(date.today(),t1)
    ct2 = datetime.combine(date.today(),t2)
    td = (ct1-ct2).total_seconds()
    return td
    
def _closest(l1,l2,res,max_delta,recycle=False,future_only=False):
    if len(l1)==0 or len(l2)==0:
        return res
    p = None
    if not future_only:
        p = product(l1, l2)
    else:
        p = [x for x in list(product(l1,l2)) if x[0]<=x[1]]
        if len(p)==0:
            return res
        
    cl = min(p, key=lambda t: abs(get_delta(t[0],t[1])))
    if abs(get_delta(cl[0],cl[1]))>max_delta:
        for x in l1:
            res.append((x,0))
        return res
    
    else:
        res.append(cl)
        t1 = [x for x in l1]
        t1.remove(cl[0])
        t2 = [x for x in l2]
        if not recycle:
            t2.remove(cl[1])
        _closest(t1,t2,res,max_delta,recycle,future_only)
        
    
def print_res(res):
    for r in res:
        try:
            print(r[0].strftime(format),r[1].strftime(format))
        except:
            print(r[0].strftime(format),None)

# now for each day and stop we need to couple datapoints and get some data out:
# 1. number missed
# 2. total time off from the schedule

closest_df = pd.DataFrame([])
closest_all_df = pd.DataFrame([])

for day,v in days.items():
    for sid,v2 in v.items():
        # couple times
        res = []
        _closest(v2["scheduled"],v2["observed"],res,600)
        
        res2 = []
        for x0,x1 in res:
            t1 = datetime.combine(day,x0)
            t2 = "1-1-1 01:01:01"
            t2a = "1-1-1 01:01:01"
            if x1 != 0:
                t2 = datetime.combine(day,x1)
                t2a = datetime.combine(day,x1)
            else:
                # what is the closest time in future?
                tr = []
                _closest([x0],v2["observed"],tr,100000000,True,True)
                if len(tr)>0:
                    t2a = datetime.combine(day,tr[0][1])
                else:
                    # nothing found add 1 hour
                    t2a = t1 + timedelta(hours=1)
            res2.append((t1,t2,t2a))
        
        tmp = pd.DataFrame(res2)
        tmp.columns = ["scheduled","observed","observed_all"]
        tmp["stop"] = sid2name[sid]
        closest_df = pd.concat([closest_df,tmp],axis=0)

# remove first and last dates
closest_df = closest_df[~(closest_df['scheduled'].dt.date==min(closest_df['scheduled'].dt.date))&
                        ~(closest_df['scheduled'].dt.date==max(closest_df['scheduled'].dt.date))].reset_index(drop=True)
closest_df.to_csv(log_fname+".closest.tsv",sep="\t",index=False)
closest_df.head()

,scheduled,observed,observed_all,stop
0,2022-11-12 18:40:00,2022-11-12 18:40:20,2022-11-12 18:40:20,Broadway
1,2022-11-12 20:20:00,2022-11-12 20:20:21,2022-11-12 20:20:21,Broadway
2,2022-11-12 07:25:00,2022-11-12 07:25:29,2022-11-12 07:25:29,Broadway
3,2022-11-12 08:15:00,2022-11-12 08:15:30,2022-11-12 08:15:30,Broadway
4,2022-11-12 00:30:00,2022-11-12 00:30:30,2022-11-12 00:30:30,Broadway


In [141]:
rlog = log.copy(deep=True)
rlog.replace({"sid":sid2name},inplace=True)
rlog = rlog[["sid","vid","time","dist"]]

rlog = rlog[~(rlog['time'].dt.date==min(rlog['time'].dt.date))&
            ~(rlog['time'].dt.date==max(rlog['time'].dt.date))].reset_index(drop=True)
rlog.to_csv(log_fname+".edited.tsv",sep="\t",index=False)
rlog

,sid,vid,time,dist
0,Broadway,4019315,2022-11-12 00:00:02-05:00,4277.045330
1,Interfaith,4019315,2022-11-12 00:00:02-05:00,5.893331
2,Broadway,4019321,2022-11-12 00:00:12-05:00,6367.668294
3,Interfaith,4019321,2022-11-12 00:00:12-05:00,10090.864038
4,Broadway,4019315,2022-11-12 00:00:12-05:00,4270.014428
...,...,...,...,...
634329,Interfaith,4019381,2022-11-17 23:59:48-05:00,5.893331
634330,Broadway,4019381,2022-11-17 23:59:52-05:00,4277.045330
634331,Interfaith,4019381,2022-11-17 23:59:52-05:00,5.893331
634332,Broadway,4019381,2022-11-17 23:59:58-05:00,4277.045330
